Sacamos la esperanza de vida a los 65 por municipio como indicador de longevidad. Luego analizaremos si la diferencia en la esperanza de vida entre municipios correlacionado con algún factor.

*Fuente*: 

* *Esperanza Vida 65a*: https://www.ine.es/dynt3/inebase/index.htm?padre=1153&capsel=1153

* *Qx por municipio*: https://www.ine.es/jaxiT3/Tabla.htm?t=30686&L=0

In [33]:
import pandas as pd
file_ev65 = 'esperanza_vida_65a.csv'
file_qx = 'qx_municipio_sexo_anyo.csv'

Sacamos el codine:

In [34]:
edv_65 = pd.read_csv(f'input/{file_ev65}',sep=";", decimal=",")
edv_65['codine'] = edv_65.Municipios.str.split(' ',expand=True)[0]
edv_65['municipio'] = edv_65.Municipios.str.split(' ',expand=True)[1]

In [35]:
qx = pd.read_csv(f'input/{file_qx}',sep=";", decimal=",")
qx['codine'] = qx.Municipios.str.split(' ',expand=True)[0]
qx['municipio'] = qx.Municipios.str.split(' ',expand=True)[1]

Simplificamos el literal del sexo:

In [36]:
sexo = {'Ambos sexos':'tot', 'Hombres':'H', 'Mujeres':'M'}
edv_65.Sexo = edv_65.Sexo.map(sexo)
qx.Sexo = qx.Sexo.map(sexo)
qx.rename(columns={"Grupo de edad":"gr_edad"}, inplace=True)

In [37]:
def text_slit(text):
    if text=='Total edades':
        return "tot"
    elif text=='90 y más años':
        return "90_a_110"
    elif text=='0 años':
        return "0_a_0"
    else:
        return "_a_".join([t for t in text.split(' ') if t.isnumeric()])
fmtEdades = dict([(gr, text_slit(gr)) for gr in qx.gr_edad.unique()])

qx.gr_edad = qx.gr_edad.map(fmtEdades)

Pivotamos los resultado para dejar como pk el codine

In [53]:
pv_edv_65 = edv_65.pivot(index='codine', columns=['Periodo','Sexo'], values='Total')
pv_edv_65.columns = ['edv65_' + c[1]+'_'+str(c[0]) for c in pv_edv_65.columns]
pv_edv_65.head(2)

,edv65_tot_2021,edv65_tot_2020,edv65_tot_2019,edv65_tot_2018,edv65_tot_2017,edv65_tot_2016,edv65_tot_2015,edv65_tot_2014,edv65_H_2021,edv65_H_2020,...,edv65_H_2015,edv65_H_2014,edv65_M_2021,edv65_M_2020,edv65_M_2019,edv65_M_2018,edv65_M_2017,edv65_M_2016,edv65_M_2015,edv65_M_2014
codine,,,,,,,,,,,,,,,,,,,,,
01059,21.91,22.07,22.09,22.13,21.95,21.68,21.83,21.70,19.55,19.77,...,19.74,19.60,24.10,24.20,24.11,24.10,23.98,23.68,23.71,23.59
02003,20.43,20.40,20.32,21.14,21.00,20.91,20.72,20.72,18.25,18.24,...,18.66,18.79,22.38,22.33,22.21,22.92,22.77,22.70,22.51,22.37


In [54]:
qx_agg = pd.concat([ qx[qx.Periodo<=2019],
                    qx[qx.Periodo<=2019].groupby(['codine','Sexo', 'gr_edad'])["Total"].mean().reset_index().assign(Periodo="tot")]
                    )
pv_qx = qx_agg.pivot(index='codine', columns=['Periodo','Sexo', 'gr_edad'], values='Total')
pv_qx.columns = ['qx_' + c[1]+'_'+str(c[0])+'_'+str(c[2]) for c in pv_qx.columns]
pv_qx.head(2)

,qx_tot_2019_tot,qx_tot_2018_tot,qx_tot_2017_tot,qx_tot_2016_tot,qx_tot_2015_tot,qx_tot_2014_tot,qx_tot_2019_0_a_0,qx_tot_2018_0_a_0,qx_tot_2017_0_a_0,qx_tot_2016_0_a_0,...,qx_tot_tot_55_a_59,qx_tot_tot_5_a_9,qx_tot_tot_60_a_64,qx_tot_tot_65_a_69,qx_tot_tot_70_a_74,qx_tot_tot_75_a_79,qx_tot_tot_80_a_84,qx_tot_tot_85_a_89,qx_tot_tot_90_a_110,qx_tot_tot_tot
codine,,,,,,,,,,,,,,,,,,,,,
01059,8.56,8.11,8.16,8.25,7.95,7.85,2.60,2.04,1.39,1.64,...,4.386667,0.090000,5.948333,8.980000,13.486667,21.808333,42.706667,84.108333,188.268333,8.146667
02003,8.64,7.36,7.34,7.38,7.38,7.25,1.83,2.17,1.35,1.51,...,3.913333,0.051667,6.765000,9.088333,15.130000,29.290000,51.713333,98.821667,206.308333,7.558333


Unimos las tablas y guardamos el resultado:

In [57]:
df_edv_qx = pv_qx.join(pv_edv_65)

In [58]:
#pv_edv_65.to_csv('output/pv_edv_65.csv',index=True)
df_edv_qx.to_csv('output/edv_qx.csv',index=True)